# Data Wrangling Challenge
### Pull and manipulate the API data

The point of this exercise is to try data enrichment with data from external APIs. We are going to take data about car crashes in Monroe County, Indiana from 2003 to 2015 and try to figure out the weather during the accident and how many bars there are in the area. We will work with two different APIs during this challenge:

- Foursquare API
- Visual Crossing API

We will try to find correlations between the severity of crash and weather/number of bars in the area. To indicate the severity of a crash, we will use column `Injury Type`.

## Data

The data for this exercise can be found [here](https://drive.google.com/file/d/1_KF9oIJV8cB8i3ngA4JPOLWIE_ETE6CJ/view?usp=sharing).

Just run the cells below to get your data ready. Little help from us.


In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import os
import foursquare
import requests

In [3]:
data = pd.read_csv("data/monroe-county-crash-data2003-to-2015.csv", encoding="unicode_escape")
# ========================
# preparing data
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['Latitude'].astype(str) + ',' + data['Longitude'].astype(str)
data = data[data['ll'] != '0.0,0.0']
print(data.shape)
data.head(20)

(49005, 13)


,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,ll
0,902363382,2015,1,5,Weekday,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015,1,6,Weekday,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015,1,6,Weekend,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015,1,7,Weekend,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015,1,7,Weekend,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"
5,902364664,2015,1,6,Weekday,1800.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,BURKS & WALNUT,39.126670,-86.531370,"39.12666969,-86.53136998"
6,902364682,2015,1,6,Weekday,1200.0,2-Car,No injury/unknown,DRIVER DISTRACTED - EXPLAIN IN NARRATIVE,SOUTH CURRY PIKE LOT 71,39.150825,-86.584899,"39.150825,-86.584899"
7,902364683,2015,1,6,Weekday,1400.0,1-Car,Incapacitating,ENGINE FAILURE OR DEFECTIVE,NORTH LOUDEN RD,39.199272,-86.637024,"39.19927216,-86.63702393"
8,902364714,2015,1,7,Weekend,1400.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,LIBERTY & W 3RD,39.164610,-86.579130,"39.16461021,-86.57913007"
9,902364756,2015,1,7,Weekend,1600.0,1-Car,No injury/unknown,RAN OFF ROAD RIGHT,PATTERSON & W 3RD,39.163440,-86.551280,"39.16344009,-86.55128002"


In [4]:
data['Injury Type'].value_counts()

No injury/unknown     37467
Non-incapacitating    10427
Incapacitating         1003
Fatal                   108
Name: Injury Type, dtype: int64

# Foursquare API

Foursquare API documentation is [here](https://developer.foursquare.com/)

1. Start a foursquare application and get your keys.
2. For each crash, create the function **get_venues** that will pull bars in the radius of 5km around the crash

#### example
`get_venues('48.146394, 17.107969')`

3. Find a relationship (if there is any) between number of bars in the area and severity of the crash.

HINTs: 
- check out python package "foursquare" (no need to send HTTP requests directly with library `requests`)
- **categoryId** for bars and nightlife needs to be found in the [foursquare API documentation](https://developer.foursquare.com/docs/api-reference/venues/search/)

In [ ]:
#set the keys
foursquare_id = os.environ[" "]
foursquare_secret = os.environ[" "]
VERSION = '20200130'

In [ ]:
# Construct the client object
client = foursquare.Foursquare(client_id=foursquare_id, client_secret=foursquare_secret, redirect_uri='http://fondu.com/oauth/authorize')

# pull the ll column out of the dataframe into a list
latlon = []
for i in data['ll']:
    latlon.append(i)

# 'Bar' CatID is 4bf58dd8d48988d116941735
# search for a venue
bars = client.venues.search(params={'query': 'Bar', 'll': '39.15920668,-86.52587356', 'radius': 5000, 'limit': 50})
bar_list = []
for bar in bars['venues']:
    bar_list.append(bar['name'])
#bar_list

In [ ]:
# build function
def get_venues(coords):
    bars = client.venues.search(params={'query': 'Bar', 'll': coords, 'radius': 5000, 'limit': 50})
    bar_list = []
    for bar in bars['venues']:
        bar_list.append(bar['name'])
    return len(bar_list)

In [ ]:
incapacitating = '39.19927216,-86.63702393'
get_venues(incapacitating)

In [ ]:
for element in data['ll']:
    data['count_bars'] = get_venues(element)

In [ ]:
df = data[['Injury Type','count_bars']]
df.groupby('Injury Type').agg('mean')

# Visual Crossing API

Virtual Crossing API documentation is [here](https://www.visualcrossing.com/resources/documentation/)

1. Sign up for FREE api key if you haven't done that before.
2. For each crush, get the weather for the location and date.
3. Find a relationship between the weather and severity of the crash.

Hints:

* randomly sample only 250 or so (due to API limits), or pull weather only for smaller sample of crashes
* for sending HTTP requests check out "requests" library [here](http://docs.python-requests.org/en/master/)


In [193]:
import requests
import time
import numpy as np

api_key = os.environ["<>"]

In [6]:
data[:1]

,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,ll
0,902363382,2015,1,5,Weekday,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"


In [ ]:
df = data.copy()
df['Date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2) + '-' + df['Day'].astype(str).str.zfill(2)
date1 = df['Date'][0]
location = df['ll'][0]


url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{date1}?unitGroup=metric&key={api_key}&include=obs%2Calerts'

#weather = pd.read_json(url)
#weather.head()
res = requests.get(url)
weather_snapshot = res.json()
weather_snapshot

In [ ]:
weatherdf = df[['Injury Type','ll','Date']].copy() # remove .head() when ready to test the full dataset
weatherdf['temp'] = np.nan
weatherdf['precip'] = np.nan
weatherdf['snow'] = np.nan
weatherdf['visibility'] = np.nan
weatherdf

In [ ]:
for row in range(len(weatherdf)):
    try:
        # grab location and date from the crash information
        location = weatherdf.loc[row,'ll']
        date1 = weatherdf.loc[row,'Date']
        # run get request
        url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{date1}?unitGroup=metric&key={api_key}&include=obs%2Calerts'
        res = requests.get(url)
        weather_snapshot = res.json()
        # index the weather api's json for information within date set
        weather_snapshot = weather_snapshot['days'][0]
        # apply weather info into df
        weatherdf.loc[row,'temp'] = weather_snapshot['temp']
        weatherdf.loc[row,'precip'] = weather_snapshot['precip']
        weatherdf.loc[row,'snow'] = weather_snapshot['snow']
        weatherdf.loc[row,'visibility'] = weather_snapshot['visibility']
    except:
        continue

In [ ]:
for row in range(len(dfloc)):
    dftest.loc[row,'temp'] = row
dftest

In [ ]:
df = data.copy()
df['Date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2) + '-' + df['Day'].astype(str).str.zfill(2)
date1 = df['Date'][0]
location = df['ll'][0]

url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{date1}?unitGroup=metric&key={api_key}&include=obs%2Calerts'
res = requests.get(url)
weather_snapshot = res.json()
weather_snapshot = weather_snapshot['days'][0]
weather_snapshot['temp']